In [11]:
# ---- Offline, deterministic market + option loader (no network, no project imports) ----
from pathlib import Path
import json
import math
import numpy as np
import pandas as pd

START, END = "2015-01-01", "2025-01-01"
RISK_FREE_RATE = 0.02
VOL_WINDOW = 20

# Deterministic RNG for any synthetic generation
_DEFAULT_RNG = np.random.default_rng(42)


def _read_csv_flexible(path: str) -> pd.DataFrame:
    """Load CSV and parse first column as timestamp index, robust to 'Unnamed: 0' or named columns."""
    df = pd.read_csv(path, index_col=0, parse_dates=True)
    df.index.name = "timestamp"
    return df.sort_index()


def _load_from_dataset_cards(prefer_split: str = "train"):
    """Read market/option from the newest runs/*/dataset_card.json entry with rows > 0.
    Returns (market_df, option_df, (start, end)) or (None, None, None).
    """
    runs_dir = Path("runs")
    cards = sorted(runs_dir.glob("*/dataset_card.json"), key=lambda p: p.stat().st_mtime, reverse=True)
    for cp in cards:
        try:
            entries = json.loads(cp.read_text())
        except Exception:
            continue
        windows = {}
        for e in entries:
            if e.get("split") != prefer_split or e.get("rows", 0) <= 0:
                continue
            win = (e.get("start"), e.get("end"))
            windows.setdefault(win, {})[e.get("kind")] = e.get("file")
        for (ws, we), kinds in sorted(windows.items()):
            if "market" in kinds:
                m = _read_csv_flexible(kinds["market"]) if kinds.get("market") else None
                o = _read_csv_flexible(kinds["option"]) if kinds.get("option") else None
                if m is not None and not m.empty:
                    return m, o, (ws, we)
    return None, None, None


def _find_runs_csv_pair():
    """Heuristic: scan runs/* for '*market*.csv' and corresponding '*option*.csv' in the same folder.
    Robust to malformed CSVs (skips on read errors).
    """
    runs_dir = Path("runs")
    if not runs_dir.exists():
        return None, None, None
    run_dirs = sorted([p for p in runs_dir.glob("*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
    for rd in run_dirs:
        market_files = sorted([p for p in rd.glob("*.csv") if "market" in p.name])
        option_files = sorted([p for p in rd.glob("*.csv") if "option" in p.name])
        if market_files:
            try:
                m = _read_csv_flexible(str(market_files[0]))
            except Exception:
                continue
            o = None
            if option_files:
                try:
                    o = _read_csv_flexible(str(option_files[0]))
                except Exception:
                    o = None
            if m is not None and not m.empty:
                win = (str(m.index.min().date()), str(m.index.max().date()))
                return m, o, win
    return None, None, None


def _gbm_synthetic(n_days: int = 300, seed: int = 42) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    mu, sigma = 0.08, 0.15
    dt = 1.0 / 252.0
    r = rng.normal(mu * dt, sigma * math.sqrt(dt), size=n_days)
    prices = 100.0 * np.exp(np.cumsum(r))
    idx = pd.date_range(START, periods=n_days, freq="B")
    df = pd.DataFrame({
        "Open": prices,
        "High": prices * (1 + np.abs(rng.normal(0, 0.001, n_days))),
        "Low": prices * (1 - np.abs(rng.normal(0, 0.001, n_days))),
        "Close": prices,
        "Volume": rng.integers(1_000_000, 10_000_000, n_days),
    }, index=idx)
    # Returns and annualized daily vol
    df["Returns"] = df["Close"].pct_change()
    df["Volatility"] = df["Returns"].rolling(VOL_WINDOW).std() * math.sqrt(252)
    return df.dropna()


# Minimal Black–Scholes pricer (no SciPy dependency)
_DEF_EPS = 1e-12


def _norm_pdf(x: np.ndarray) -> np.ndarray:
    return (1.0 / math.sqrt(2.0 * math.pi)) * np.exp(-0.5 * np.asarray(x) ** 2)


def _norm_cdf(x: np.ndarray) -> np.ndarray:
    # Use math.erf applied elementwise to avoid SciPy
    x_arr = np.asarray(x, dtype=float)
    erf_vec = np.vectorize(math.erf)
    return 0.5 * (1.0 + erf_vec(x_arr / math.sqrt(2.0)))


def _bs_price_delta(S: np.ndarray, K: float, T: float, sigma: np.ndarray, r: float = RISK_FREE_RATE):
    S = np.asarray(S, dtype=float)
    sigma = np.asarray(sigma, dtype=float)
    T = float(T)
    # Guardrails
    sigma = np.maximum(sigma, 1e-6)
    S = np.maximum(S, _DEF_EPS)
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * _norm_cdf(d1) - K * math.exp(-r * T) * _norm_cdf(d2)
    call_delta = _norm_cdf(d1)
    return call_price, call_delta


def _compute_option_df(market_df: pd.DataFrame, strike_price: float = None, T: float = 1.0 / 252.0) -> pd.DataFrame:
    """Compute call option price/delta per timestamp from market_df using rolling vol."""
    if strike_price is None:
        strike_price = float(market_df["Close"].iloc[0])
    sigma = market_df["Volatility"].fillna(market_df["Volatility"].median()).to_numpy()
    S = market_df["Close"].to_numpy()
    prices, deltas = _bs_price_delta(S, strike_price, T, sigma, r=RISK_FREE_RATE)
    option_df = pd.DataFrame({
        "option_price": prices,
        "delta": deltas,
        "volatility": sigma,
        "strike": strike_price,
    }, index=market_df.index)
    return option_df


# Load priority: dataset_card.json -> runs/* CSVs -> synthetic GBM
market, option, window = _load_from_dataset_cards(prefer_split="train")
if market is None:
    m2, o2, win2 = _find_runs_csv_pair()
    market, option, window = m2, o2, win2

if market is None:
    market = _gbm_synthetic(n_days=400, seed=42)
    window = (str(market.index.min().date()), str(market.index.max().date()))

# Ensure returns/vol present (if loaded from CSVs that lacked them)
if "Returns" not in market.columns:
    market["Returns"] = market["Close"].pct_change()
if "Volatility" not in market.columns:
    market["Volatility"] = market["Returns"].rolling(VOL_WINDOW).std() * math.sqrt(252)
market = market.dropna()

# Ensure we have option data; if missing or misaligned, compute locally
if option is None or option.empty or not market.index.equals(option.index):
    option = _compute_option_df(market)

print(f"Loaded market rows: {len(market):,}, option rows: {len(option):,}, window: {window}")


Loaded market rows: 380, option rows: 380, window: ('2015-01-29', '2016-07-13')


In [12]:
# Use the loaded window and align frames; avoid project configs
start, end = window
m = market.loc[str(start):str(end)]
o = option.loc[m.index]

# Guard against emptiness early (episode_length=60)
EP_LEN = 60
min_len = EP_LEN + 1
if len(m) < min_len or len(o) < min_len:
    raise RuntimeError(f"Not enough rows in the selected window ({len(m)}). Need at least {min_len}.")

In [13]:
# Self-contained offline hedging harness: simulate delta hedging and compute PnL + CVaR
import numpy as np
import pandas as pd

# Parameters matching env defaults
EP_LEN = 60
POSITION_SCALE = 1000
TC_BPS = 1.0  # 0.0001 fraction -> 1 bps; we'll allow explicit bps here


def simulate_delta_hedge(m: pd.DataFrame, o: pd.DataFrame, episode_length: int = EP_LEN,
                          position_scale: float = POSITION_SCALE, tc_bps: float = TC_BPS,
                          seed: int = 42):
    rng = np.random.default_rng(seed)
    assert len(m) == len(o)
    n = len(m)
    # Single continuous episode over the window
    hedge_position = 0.0
    portfolio_value = 0.0
    pnl_steps = []
    for t in range(min(n - 1, episode_length)):
        current_price = float(m.iloc[t]["Close"]) 
        next_price = float(m.iloc[t + 1]["Close"]) 
        current_option = o.iloc[t]
        next_option = o.iloc[t + 1]

        option_pnl = float(next_option["option_price"] - current_option["option_price"]) 
        # Hedge target equals -delta (simple delta hedger)
        target_position = -float(current_option["delta"]) * float(position_scale)
        position_change = target_position - hedge_position
        tc_rate = float(tc_bps) / 10000.0
        transaction_costs = abs(position_change) * current_price * tc_rate
        hedge_pnl = -hedge_position * (next_price - current_price)

        total_pnl = option_pnl + hedge_pnl - transaction_costs
        pnl_steps.append(total_pnl)

        portfolio_value += total_pnl
        hedge_position = target_position

    return np.array(pnl_steps)


# CVaR utility (left tail, losses negative)
def cvar(values, alpha: float = 0.95):
    arr = np.asarray(values, dtype=float)
    if arr.size == 0:
        return 0.0
    k = max(1, int(np.floor((1.0 - alpha) * arr.size)))
    tail = np.sort(arr)[:k]
    return float(tail.mean())


episode_pnl = simulate_delta_hedge(m, o, episode_length=EP_LEN, position_scale=POSITION_SCALE, tc_bps=TC_BPS)

cvar_95 = cvar(episode_pnl, 0.95)
print(f"Episode steps: {len(episode_pnl)}, CVaR95: {cvar_95:.6f}, total PnL: {episode_pnl.sum():.6f}")


Episode steps: 60, CVaR95: -1436.485928, total PnL: 4721.723563


In [14]:
# Optional: save arrays to runs/ for reproducibility (no network)
from pathlib import Path
import numpy as np

out_dir = Path("runs") / "offline_demo"
out_dir.mkdir(parents=True, exist_ok=True)
np.save(out_dir / "episode_pnl.npy", episode_pnl)
with open(out_dir / "summary.txt", "w") as f:
    f.write(f"window={window}\nrows={len(m)}\nsteps={len(episode_pnl)}\ncvar95={cvar_95}\n")
print(f"Saved PnL and summary to {out_dir}")


Saved PnL and summary to runs/offline_demo


In [15]:
# Disabled: project imports and network-bound data are not used in offline mode.
print("Using offline harness above. This cell is disabled.")

Using offline harness above. This cell is disabled.
